# ERA5 Volumetric Soil Moisture Climatology: Visualize

In [1]:
import ee
from statistics import mean

import sys
sys.path.insert(1, '../SMAP-USDA/')  # until we package this
import visualize as vis
import water_common as common
import PIL
from PIL import Image

Loading BokehJS ...

In [2]:
# ee.Authenticate()
ee.Initialize()

In [16]:
moisture_clim_path1 = 'users/amgadellaboudy/era5_vol_soil_moisture_conus_climatology1'
moisture_clim1 = ee.ImageCollection(moisture_clim_path1)
moisture_clim_path2 = 'users/amgadellaboudy/era5_vol_soil_moisture_conus_climatology-2'
moisture_clim2 = ee.ImageCollection(moisture_clim_path2)

In [17]:
palette_name = 'BrBG'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, maximum=0.5, minimum=0)

In [5]:
vis_params = {
    'min': 0, 'max': 0.5, 'dimensions': 512,
    'palette': palette}

## Static Image

In [20]:
variable ='potential_sfc_moisture_mon_clim_cms'
mm = 1
img1 = (
    moisture_clim1
    .filter(ee.Filter.eq('month', mm))
    .filter(ee.Filter.eq('variable', variable)).first())
img2 = (
    moisture_clim2
    .filter(ee.Filter.eq('month', mm))
    .filter(ee.Filter.eq('variable', variable)).first())

In [21]:
display(vis.Image(url = img1.getThumbURL(vis_params)))   #visualization for first layer moisture, 0-7 cm
display(vis.Image(url = img2.getThumbURL(vis_params)))  #visualization for second layer moisture, 7-28 cm


## Folium Layers

In [8]:
box_corners = common.bboxes()['conus'].toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

In [23]:
the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=4, height=500)
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for mm0 in range(12):
    mm = mm0 + 1
    img = (
        moisture_clim2 
        .filter(ee.Filter.eq('month', mm))
        .filter(ee.Filter.eq('variable', variable)).first())
    the_map.add_ee_layer(img, vis_params, variable + ', ' + month_names[mm0])

vis.folium_display(the_map)

## geemap 

In [11]:
import geemap.eefolium as geemap

In [12]:
Map = geemap.Map()
landcover = ee.Image('MODIS/006/MCD12Q1/2013_01_01') \
    .select('LC_Type1') \

igbpLandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}

Map.setCenter(6.746, 46.529, 2)
Map.addLayer(landcover, igbpLandCoverVis, 'MODIS Land Cover')
Map.add_legend(builtin_legend='MODIS/006/MCD12Q1')

Map.addLayerControl()

Map